### Running the python model for MAPC's Equitable Green Stormwater Infrastructure Site Selection tool

This notebook is meant for running the "parcel ms4 model" developed for MAPC's Equitable Green Infrastructure Site Selection Tool. Data inputs can be swapped out at src > data > make_dataset.py. See GitBook documentation for more information.

In [1]:
#import packages 

import sys
sys.path.append("..")
import os
os.environ['USE_PYGEOS'] = '0'
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols 
from rasterstats import zonal_stats
import rasterio
import contextily as cx
from rasterio.plot import show
#import osmnx as ox
from affine import Affine
import rioxarray as rx
from multiprocessing import Pool, cpu_count
from shapely.ops import unary_union
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
from pathlib import Path
import fiona
import pygeos

#see all columns in tables
pd.set_option('display.max_columns', None)


%load_ext autoreload
%autoreload 2

In [2]:
#import parcel model function 
from src.features.parcel_model import parcel_ms4_model

Reading in municipal and regional boundaries...
Reading in land parcel database data...
Reading in additional data layers...
Wetlands...
Watersheds...
Wellhead protection areas...
Activity use limitation areas...
ParkServe data...


In [5]:
#run parcel based model for each muni in MAPC

from src.data.make_dataset import munis

processed_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Output\\Parcels_20231107"

'''
for town_name in munis['municipal'].unique():
    parcel_ms4_model(town_name, processed_path)
'''


#run suitability model for each town in list
for town_name in munis['municipal'].unique():
    if town_name in os.listdir(processed_path): #set to skip any towns for which data has already been produced
        pass
    else:
        parcel_ms4_model(town_name, processed_path)

# MERGE SHAPEFILES 
from src.data.make_dataset import munis

model_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Model"

filelist = []

# add all shapefiles - takes about 30 minutes
for root, folder, files in os.walk(processed_path):
    for file in files:
        for muni in munis['municipal'].unique():
            if muni in file:
                if file.endswith('.shp'):
                    fullname = os.path.join(root, file)
                    filelist.append(fullname)


merged_gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in filelist], 
                        ignore_index=True), crs=gpd.read_file(filelist[0]).crs)

#export
merged_gdf.to_file(model_path + '\\Merged_MS4_Parcels.shp')

Starting on Lexington at 2023-11-07 17:18:14...
Starting on Everett at 2023-11-07 17:23:06...
Starting on Bellingham at 2023-11-07 17:30:27...
Starting on Millis at 2023-11-07 17:33:07...
Starting on Essex at 2023-11-07 17:34:44...
Starting on Medford at 2023-11-07 17:37:06...
Starting on Concord at 2023-11-07 17:40:36...
Starting on Southborough at 2023-11-07 17:43:07...
Starting on Wilmington at 2023-11-07 17:44:49...
Starting on Gloucester at 2023-11-07 17:47:55...
Starting on Hamilton at 2023-11-07 17:55:04...
Starting on Dover at 2023-11-07 17:56:23...
Starting on Walpole at 2023-11-07 18:00:26...
Starting on Westwood at 2023-11-07 18:03:58...
Starting on Canton at 2023-11-07 18:06:02...
Starting on Sharon at 2023-11-07 18:09:22...
Starting on Norwood at 2023-11-07 18:12:41...
Starting on Stoughton at 2023-11-07 18:16:17...
Starting on Hingham at 2023-11-07 18:19:44...
Starting on Saugus at 2023-11-07 18:24:12...
Starting on Quincy at 2023-11-07 18:28:01...
Starting on Reading at 

In [6]:
# MERGE SHAPEFILES 
from src.data.make_dataset import munis

model_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Model"

filelist = []

# add all shapefiles - takes about 30 minutes
for root, folder, files in os.walk(processed_path):
    for file in files:
        for muni in munis['municipal'].unique():
            if muni in file:
                if file.endswith('.shp'):
                    fullname = os.path.join(root, file)
                    filelist.append(fullname)


merged_gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in filelist], 
                        ignore_index=True), crs=gpd.read_file(filelist[0]).crs)

#export
merged_gdf.to_file(model_path + '\\Merged_MS4_Parcels.shp')


---

If some (but not all) municipalities need to be updated, recommend running the script below

In [ ]:
#rerun for a single muni and then replace those parcels in the old model with the new one

from src.features.parcel_model import parcel_ms4_model
from src.data.make_dataset import munis


processed_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Output\\Parcels"
model_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Model"


#list one or more munis
update_munis = ['Boston', 'Bolton']

#run script

for municipality in update_munis:
    parcel_ms4_model(municipality, processed_path)


#rejoin all municipalities in "processed_path"
filelist = []

# add all shapefiles - takes about 30 minutes
for root, folder, files in os.walk(processed_path):
    for file in files:
        for muni in munis['municipal'].unique():
            if muni in file:
                if file.endswith('.shp'):
                    fullname = os.path.join(root, file)
                    filelist.append(fullname)


merged_gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in filelist], 
                        ignore_index=True), crs=gpd.read_file(filelist[0]).crs)

#export
merged_gdf.to_file(model_path + '\\Merged_MS4_Parcels.shp')


In [3]:
from src.features.parcel_model import parcel_ms4_model

processed_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Output\\Parcels"


#list one or more munis
update_munis = ['Arlington']

#run script

for municipality in update_munis:
    parcel_ms4_model(municipality, processed_path)

#F_752770_2973342


Starting on Arlington at 2023-11-07 17:08:51...
corrected
